In [135]:
import pandas as pd
import numpy as np
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

In [136]:
from src.data.provide_data import get_model_data
from src.models.evaluate import custom_classification_report, custom_lazy_report



In [137]:
X_train, y_train = get_model_data(filename = "Train")
X_valid, y_valid = get_model_data(filename = "Valid")
X_test, y_test = get_model_data(filename = "Test")

cat = [X_train.columns.get_loc(i) for i in ["Team","Div","Opponent"]]

Train 5246
Valid 1952
Test 1952


In [138]:
def add_bets(y,X,columns=['BW_Team','BW_Draw','BW_Opponent']):
    y_bets = pd.concat([y,X[columns]],axis=1)
    return y_bets

In [139]:
y_train_bets = np.array(add_bets(y=y_train,X=X_train,columns=['BW_Team','BW_Draw','BW_Opponent']))
y_test_bets = np.array(add_bets(y=y_test,X=X_test,columns=['BW_Team','BW_Draw','BW_Opponent']))
y_valid_bets = np.array(add_bets(y=y_valid,X=X_valid,columns=['BW_Team','BW_Draw','BW_Opponent']))

In [140]:
features = X_train.columns

In [141]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler object
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Scale training and test data
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB(alpha=10,binarize=0.5)
bnb.fit(X_train,y=y_train)
y_pred = bnb.predict(X_valid)

In [142]:
import pandas as pd

def get_column_indices(df, column_names):
    """
    Returns the indices of the given column names within the dataframe.
    
    Args:
    - df (pd.DataFrame): the dataframe
    - column_names (list of str): the names of the columns to get indices for
    
    Returns:
    - indices (list of int): the indices of the columns within the dataframe
    """
    indices = []
    for col_name in column_names:
        try:
            indices.append(df.columns.get_loc(col_name))
        except KeyError:
            print(f"Column name '{col_name}' not found in dataframe.")
    return indices


In [143]:
ind  = get_column_indices(df = pd.DataFrame(X_valid,columns=features), column_names = ["B365_Team","B365_Draw","B365_Opponent"])

# Transformer

## Pretrainer

In [ ]:

import torch

from pytorch_tabnet.pretraining import TabNetPretrainer

# Define pretrainer model architecture
pretrainer = TabNetPretrainer(
optimizer_fn=torch.optim.Adam,
optimizer_params=dict(lr=2e-2),
mask_type="entmax"
)

# Train pretrainer model on training data
pretrainer.fit(
    X_train=X_train,
    eval_set=[X_valid],
    max_epochs=1000,
    patience=30,
    pretraining_ratio=0.8,
    batch_size= 64
)


In [ ]:
from src.utils.utils import save_as_pickle

save_as_pickle(pretrainer,'pretrainer.pkl')

In [144]:
from src.utils.utils import load_from_pickle

ImportError: cannot import name 'load_from_pickle' from 'src.utils.utils' (c:\Users\Robert\Documents\Projekte\dev\sport_betting\src\utils\utils.py)

In [ ]:
from src.utils.utils import load_from_pickle

load_from_pickle(CONFIG.DATA_FOLDER_MODELS+'pretrainer.pkl')

# LossFunction TESTS

In [ ]:
from src.models.loss_function import PrecisionLoss, TotalValueLoss


import torch
from pytorch_tabnet.tab_model import TabNetClassifier
n_d = 8
n_a = 8
n_steps = 3
gamma = 1
lambda_sparse = 0.001
lr = 2e-2
batch_size = 128
max_epochs = 1000
# Define a custom loss function

import torch
import torch.nn as nn

model = TabNetClassifier(n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=lr), mask_type='entmax', device_name='cuda' if torch.cuda.is_available() else 'cpu')

# Define and train the model with the custom loss function
model.fit(X_train=X_train, y_train=y_train,     eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'], eval_metric=['balanced_accuracy','auc'],batch_size=batch_size, max_epochs=max_epochs, patience=50,loss_fn=TotalValueLoss(),from_unsupervised=pretrainer)


# Eval Metric TESTS

In [ ]:
from sklearn.metrics import precision_score
from typing import List

class Precision(Metric):
    def __init__(self):
        self._name = "precision"
        self._maximize = True

    def __call__(self, y_true: List[int], y_pred: List[int]):
        precision = precision_score(y_true, y_pred[:,1]>0.5)
        return precision

model.fit(X_train=X_train, y_train=y_train,  eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],eval_metric=['auc',Precision],batch_size=batch_size, max_epochs=max_epochs, patience=50)

In [ ]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
n_d = 8
n_a = 8
n_steps = 3
gamma = 1
lambda_sparse = 0.001
lr = 2e-2
batch_size = 128
max_epochs = 1000
# Define a custom loss function

import torch
import torch.nn as nn

from functools import partial

class BetLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, output, target):
        if len(output.shape)>1:
            output_array =  torch.clamp(torch.abs(output[:,1]),min=0, max=1)
        else:
            output_array = output
        output_array = torch.clamp(torch.abs(output[:,1]),min=0, max=1)            

        if len(target.shape)>1:
            target_array = target[:,0]
        else:
            target_array = target
        target_array = torch.clamp(torch.abs(target_array),min=0, max=1)      
        #self.bets
        output_array = output_array>0.5
        target_array = target_array>0.5
        # loss = torch.sum(torch.abs(self.X_train_subset) * (torch.abs(target)-torch.abs(output)))/ torch.sum(torch.abs(self.X_train_subset))
        # precision = torch.sum((output_array == True) & (target_array == True))/target_array.size(0)
        loss = torch.sum(1/output_array[(output_array > 0.5) & (target_array != True)])/torch.sum(1/output_array[(output_array > 0.5)])
        
        return loss
    

    
ind = [i for i, x in enumerate(features) if x in ["BW_Team"]]

model = TabNetClassifier(n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=lr), mask_type='entmax', device_name='cuda' if torch.cuda.is_available() else 'cpu')

# Define and train the model with the custom loss function
# Define and train the model with the custom loss function
model.fit(X_train=X_train, y_train=y_train,  eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid',"precision"], eval_metric=['balanced_accuracy','auc',Precision],batch_size=batch_size, max_epochs=max_epochs, patience=50,loss_fn=BetLoss())


## Evaluierung

In [ ]:
model

In [ ]:
from matplotlib import pyplot as plt
clf = model
#plt.plot(clf.history['loss'], label='Validation Loss')
plt.plot(clf.history['valid_auc'], label='AUC')
plt.plot(clf.history['valid_balanced_accuracy'], label='Accuracy')

plt.legend()
plt.show()

from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
r = classification_report(y_pred=y_pred,y_true=y_test)
print(r)

In [ ]:
sum(y_test_bets[:,1][y_pred&y_test])-sum(y_pred)

## Erklärbarkeit

In [ ]:
import numpy as np
f_i = np.round(pd.Series(clf.feature_importances_),3)
f_i.index = features
f_i = f_i.sort_values(ascending=False)
print(f_i)

In [ ]:
"BW_opponent_odd_pred","B365_Opponent","Avg_Opponent","IW_Team","Span_Draw"      

In [ ]:
sum(f_i>0.01)